# Analisi dei dati con Apache Hive e Pig

Utilizzeremo ``hive`` come linguaggio di query per la creazione del database relativo ai passeggeri del Titanic e per trasformare i dati in modo da consentire a Pig di accedervi correttamente. Il dataset ``csv`` del Titanic ha una serie di campi vuoti in alcune rghe e non è _qualified_ cioè non garantisce che tutti i campi testuali siano racchiusi da apici.

Successivamente useremo ``pig`` per eseguire la nostra analisi sul tasso di sopravvivenza per classe e per genere dei passeggeri.

## Installazione e configurazione

Entrambi i pacchetti possono essere installati scaricandoli dal sito di [Apache Foundation](https://downloads.apache.org/). ``pig`` non richiede particolari configurazioni perché è un client che si connette direttamente alla URL del namenode HDFS [hdfs://localhost:8020]() ovvero [hdfs://localhost:9000]().

D'altro canto ``hive`` deve essere configurato per quanto riguarda quantomeno la creazione del metastore e l'accesso al server hive. Assumeremo un metastore implementato come database MySQL.

Si consiglia di installare `hive`all'interno della home directory dell'utente `hadoop` proprietario di Apache hadoop.

### Creazione del metastore

Si creerà undatabase ``metastore`` con utente ``hiveuser/password``.

```sql
mysql> CREATE DATABASE metastore;
mysql> USE metastore;
mysql> CREATE USER 'hiveuser'@'localhost' IDENTIFIED BY 'password';
mysql> GRANT * ON metastore.* TO 'hiveuser'@'localhost' WITH GRANT OPTION;
mysql> FLUSH PRIVILEGES;
```

### Creazione del file di configurazione ``hive-site.xml``

Il file di configurazione dovrà almeno contenere le informazioni per l'accesso al database MySQL che implementa il metastore e le credenziali per il server. 

Si consiglia di copiare il file template e poi modificarlo:

```bash
$ export HIVE_HOME=/percorso/di/installazione/hive
$ export PATH=$PATH:$HIVE_HOME/bin
$ cd $HIVE_HOME/conf
$ cp hive-default.xml.template hive-site.xml
```

Le seguenti informazioni vanno inserite/modificate che riguardano il driver ``jdbc`` per l'accesso al database, le crdenziali del'utent del database e la posizione della directory temporanea in HDFS dove ``hive`` gestirà i suoi dati.

```xml
<property>
  <name>javax.jdo.option.ConnectionURL</name>
  <value>jdbc:mysql://localhost/metastore</value>
</property>
<property>
  <name>javax.jdo.option.ConnectionDriverName</name>
  <value>com.mysql.cj.jdbc.Driver</value>
</property>
<property>
  <name>javax.jdo.option.ConnectionUserName</name>
  <value>hiveuser</value>
</property>
<property>
  <name>javax.jdo.option.ConnectionPassword</name>
  <value>password</value>
</property>
<property>
  <name>datanucleus.fixedDatastore</name>
  <value>false</value>
</property>
<property>
    <name>hive.exec.local.scratchdir</name>
    <value>/tmp/hive</value>
    <description>Local scratch space for Hive jobs</description>
</property>
<property>
    <name>hive.downloaded.resources.dir</name>
    <value>/tmp/hive</value>
    <description>Temporary local directory for added resources in the remote file system.</description>
</property>
<property>
    <name>hive.querylog.location</name>
    <value>/tmp/hive</value>
    <description>Location of Hive run time structured log file</description>
</property>
<!-- Accesso al server hive -->
<property>
    <name>hive.server2.thrift.client.user</name>
    <value>pirrone</value>
    <description>Username to use against thrift client</description>
</property>
<property>
    <name>hive.server2.thrift.client.password</name>
    <value>pirrone</value>
    <description>Password to use against thrift client</description>
</property>
```

Successivamente, vanno create le cartelle in HFDS per l'uso di ``hive``:

```bash
$ hadoop fs -mkdir -p /user/hive/warehouse
$ hadoop fs -chmod g+w /user/hive/warehouse
$ hadoop fs -mkdir -p /tmp
$ hadoop fs -chmod g+w /tmp
$ hadoop fs -mkdir -p /tmp/hive
$ hadoop fs -chmod 777 /tmp/hive
```

A questo punto si può inizializzare lo schema del metastore con il comando ``schematool``, essendo certi che punterà al nostro database:

```bash
$ schematool -dbType mysql -initSchema -userName hiveuser -passWord password 
```

## Connessione a HDFS con ``hive``  o ``beeline``

L'accesso tramite la CLI di ``hive`` è immediato:

```bash
$ hive
hive> 
```

Per utilizzare ``beeline``, per altro consigliato, si deve attivare il server perché ci si deve connettere via rete.

```bash
$ hiveserver2 &
$ beeline
Beeline version 2.3.7 by Apache Hive
beeline>!connect jdbc:hive2://localhost:10000
Connecting to jdbc:hive2://localhost:10000
Enter username for jdbc:hive2://localhost:10000: pirrone
Enter password for jdbc:hive2://localhost:10000: *******
Connected to: Apache Hive (version 2.3.7)
Driver: Hive JDBC (version 2.3.7)
Transaction isolation: TRANSACTION_REPEATABLE_READ
0: jdbc:hive2://localhost:10000> 
````

Da questo momento possiamo operare con il linguaggio di query.


## Creazione del data set

Creiamo lo schema della tabella per ospitare i dati del dataset titanic. Useremo un SERializer/DEserializer per leggere le righe e salteremo la riga delle intestazioni.

```sql
create table passengers (id int, survived int, class int, name varchar(64), 
sex varchar(8), age float, sibsp int, parch int, ticket varchar(20), 
fare float, cabin varchar(20), embarked char(1)) 
row format serde 'org.apache.hadoop.hive.serde2.OpenCSVSerde' 
TBLPROPERTIES ("skip.header.line.count"="1");
````

Carichiamo i dati dal file system locale.

```sql
load data local inpath './Data/titanic.csv' into table passengers;
```

Creiamo lo shcema tipizzato attribuendo un tipo ai campi e impostando la tabulazione come separatore dei campi.

```sql
create table passengers_qualified (id int, survived int, class int, 
name varchar(64), sex varchar(8), age float, sibsp int, parch int, 
ticket varchar(20), fare float, cabin varchar(20), embarked char(1)) 
row format delimited fields terminated by '\t';
````

Infine carichiamo i dati nel nuovo schema. Questo implicitamente scrive un file ``passengers_qualified`` in HDFS. Questo file verrà caricato in ``pig`` per continuare l'analisi.

```sql
insert overwrite table passengers_qualified select * from passengers;
```


## Analisi dei dati in ``pig``

Iniziamo registrando una UDF (User Defined Function) codificata in Javascript per il calcolo della media.

La funzion si trova sul file ``percentage_pig.js``:

```javascript
percentage.OutputSchema = "perc:double";

function percentage(num, total){
	return num * 100.0 / total;
}
```

L'analogo Python usa i decoratori:

```python
from pig_util import outputSchema;

@outputSchema("perc: double")
def percentage(num, total):
	return num*100/total;
```

La direttiva di registrazione è:

```
register './percentage_pig.js' using javascript as myfuncs;
```

Carichiamo i dati generati da ``hive`` usando la funzione ``PigStorage()`` che gestisce le operazioni SER/DE e impostando lo schema dei dati.

```
titanic = load 'hdfs://localhost:9099/user/hive/warehouse/titanic.db/passengers_qualified' 
using PigStorage() as (id:int, survived:int, class:int, 
name:chararray, sex:chararray,age:int, sibsp:int, parch:int, ticket:chararray, 
fare:double, cabin:chararray, embarked:chararray);
```

Raggruppiamo i dati per genere e classe al fine di contare gli imbarcati per ogni tipo. Queste clausole **non eseguono** task YARN sui dati. Agiscono solo sui metadati. Le operazioni di ``dump`` materializzano gli alias generando il piano di esecuzione MapReduce.

```
embarked = group titanic by (sex,class);

embarked_totals = FOREACH embarked GENERATE $0, COUNT(titanic) as embarked:int;
```

Generiamo l'alias dei sopravvissuti e raggruppiamolo per classe e genere.

```
survived = filter titanic by survived == 1;

survived_grouped = group survived by (sex,class);
```

Eseguiamo il join sulla chiave ``(sex, class)`` di imbarcati e sopravvissuti

```
embarked_survived = JOIN survived_grouped BY $0, embarked_totals BY $0;
```

Generiamo infine i conteggi dei sopravvissuti, degli imbarcati, la percentuale e l'età media a partire dal join.

```
survived_counts = FOREACH embarked_survived GENERATE $0, COUNT(survived), $3,
COUNT(survived)*100.0/$3, AVG(survived.age);
```

Analogo con la nostra UDF

```
survived_stats = FOREACH survived_counts GENERATE $0, $1, myfuncs.percentage($1,$2), $4;
```

Salvataggio in HDFS tramite il SER/DE ``PigStorage()``

```
store survived_counts into 'hdfs://localhost:9099/user/pirrone/pig/survived_stats' using PigStorage();




# Analisi dei dati con Apache Hive e Pig

Utilizzeremo ``hive`` come linguaggio di query per la creazione del database relativo ai passeggeri del Titanic e per trasformare i dati in modo da consentire a Pig di accedervi correttamente. Il dataset ``csv`` del Titanic ha una serie di campi vuoti in alcune rghe e non è _qualified_ cioè non garantisce che tutti i campi testuali siano racchiusi da apici.

Successivamente useremo ``pig`` per eseguire la nostra analisi sul tasso di sopravvivenza per classe e per genere dei passeggeri.

## Installazione e configurazione

Entrambi i pacchetti possono essere installati scaricandoli dal sito di [Apache Foundation](https://downloads.apache.org/). ``pig`` non richiede particolari configurazioni perché è un client che si connette direttamente alla URL del namenode HDFS [hdfs://localhost:8020]() ovvero [hdfs://localhost:9000]().

D'altro canto ``hive`` deve essere configurato per quanto riguarda quantomeno la creazione del metastore e l'accesso al server hive. Assumeremo un metastore implementato come database MySQL.

Si consiglia di installare `hive`all'interno della home directory dell'utente `hadoop` proprietario di Apache hadoop.

### Creazione del metastore

Si creerà undatabase ``metastore`` con utente ``hiveuser/password``.

```sql
mysql> CREATE DATABASE metastore;
mysql> USE metastore;
mysql> CREATE USER 'hiveuser'@'localhost' IDENTIFIED BY 'password';
mysql> GRANT * ON metastore.* TO 'hiveuser'@'localhost' WITH GRANT OPTION;
mysql> FLUSH PRIVILEGES;
```

### Creazione del file di configurazione ``hive-site.xml``

Il file di configurazione dovrà almeno contenere le informazioni per l'accesso al database MySQL che implementa il metastore e le credenziali per il server. 

Si consiglia di copiare il file template e poi modificarlo:

```bash
$ export HIVE_HOME=/percorso/di/installazione/hive
$ export PATH=$PATH:$HIVE_HOME/bin
$ cd $HIVE_HOME/conf
$ cp hive-default.xml.template hive-site.xml
```

Le seguenti informazioni vanno inserite/modificate che riguardano il driver ``jdbc`` per l'accesso al database, le crdenziali del'utent del database e la posizione della directory temporanea in HDFS dove ``hive`` gestirà i suoi dati.

```xml
<property>
  <name>javax.jdo.option.ConnectionURL</name>
  <value>jdbc:mysql://localhost/metastore</value>
</property>
<property>
  <name>javax.jdo.option.ConnectionDriverName</name>
  <value>com.mysql.cj.jdbc.Driver</value>
</property>
<property>
  <name>javax.jdo.option.ConnectionUserName</name>
  <value>hiveuser</value>
</property>
<property>
  <name>javax.jdo.option.ConnectionPassword</name>
  <value>password</value>
</property>
<property>
  <name>datanucleus.fixedDatastore</name>
  <value>false</value>
</property>
<property>
    <name>hive.exec.local.scratchdir</name>
    <value>/tmp/hive</value>
    <description>Local scratch space for Hive jobs</description>
</property>
<property>
    <name>hive.downloaded.resources.dir</name>
    <value>/tmp/hive</value>
    <description>Temporary local directory for added resources in the remote file system.</description>
</property>
<property>
    <name>hive.querylog.location</name>
    <value>/tmp/hive</value>
    <description>Location of Hive run time structured log file</description>
</property>
<!-- Accesso al server hive -->
<property>
    <name>hive.server2.thrift.client.user</name>
    <value>pirrone</value>
    <description>Username to use against thrift client</description>
</property>
<property>
    <name>hive.server2.thrift.client.password</name>
    <value>pirrone</value>
    <description>Password to use against thrift client</description>
</property>
```

Successivamente, vanno create le cartelle in HFDS per l'uso di ``hive``:

```bash
$ hadoop fs -mkdir -p /user/hive/warehouse
$ hadoop fs -chmod g+w /user/hive/warehouse
$ hadoop fs -mkdir -p /tmp
$ hadoop fs -chmod g+w /tmp
$ hadoop fs -mkdir -p /tmp/hive
$ hadoop fs -chmod 777 /tmp/hive
```

A questo punto si può inizializzare lo schema del metastore con il comando ``schematool``, essendo certi che punterà al nostro database:

```bash
$ schematool -dbType mysql -initSchema -userName hiveuser -passWord password 
```

## Connessione a HDFS con ``hive``  o ``beeline``

L'accesso tramite la CLI di ``hive`` è immediato:

```bash
$ hive
hive> 
```

Per utilizzare ``beeline``, per altro consigliato, si deve attivare il server perché ci si deve connettere via rete.

```bash
$ hiveserver2 &
$ beeline
Beeline version 2.3.7 by Apache Hive
beeline>!connect jdbc:hive2://localhost:10000
Connecting to jdbc:hive2://localhost:10000
Enter username for jdbc:hive2://localhost:10000: pirrone
Enter password for jdbc:hive2://localhost:10000: *******
Connected to: Apache Hive (version 2.3.7)
Driver: Hive JDBC (version 2.3.7)
Transaction isolation: TRANSACTION_REPEATABLE_READ
0: jdbc:hive2://localhost:10000> 
````

Da questo momento possiamo operare con il linguaggio di query.


## Creazione del data set

Creiamo lo schema della tabella per ospitare i dati del dataset titanic. Useremo un SERializer/DEserializer per leggere le righe e salteremo la riga delle intestazioni.

```sql
create table passengers (id int, survived int, class int, name varchar(64), 
sex varchar(8), age float, sibsp int, parch int, ticket varchar(20), 
fare float, cabin varchar(20), embarked char(1)) 
row format serde 'org.apache.hadoop.hive.serde2.OpenCSVSerde' 
TBLPROPERTIES ("skip.header.line.count"="1");
````

Carichiamo i dati dal file system locale.

```sql
load data local inpath './Data/titanic.csv' into table passengers;
```

Creiamo lo shcema tipizzato attribuendo un tipo ai campi e impostando la tabulazione come separatore dei campi.

```sql
create table passengers_qualified (id int, survived int, class int, 
name varchar(64), sex varchar(8), age float, sibsp int, parch int, 
ticket varchar(20), fare float, cabin varchar(20), embarked char(1)) 
row format delimited fields terminated by '\t';
````

Infine carichiamo i dati nel nuovo schema. Questo implicitamente scrive un file ``passengers_qualified`` in HDFS. Questo file verrà caricato in ``pig`` per continuare l'analisi.

```sql
insert overwrite table passengers_qualified select * from passengers;
```


## Analisi dei dati in ``pig``

Iniziamo registrando una UDF (User Defined Function) codificata in Javascript per il calcolo della media.

La funzion si trova sul file ``percentage_pig.js``:

```javascript
percentage.OutputSchema = "perc:double";

function percentage(num, total){
	return num * 100.0 / total;
}
```

L'analogo Python usa i decoratori:

```python
from pig_util import outputSchema;

@outputSchema("perc: double")
def percentage(num, total):
	return num*100/total;
```

La direttiva di registrazione è:

```
register './percentage_pig.js' using javascript as myfuncs;
```

Carichiamo i dati generati da ``hive`` usando la funzione ``PigStorage()`` che gestisce le operazioni SER/DE e impostando lo schema dei dati.

```
titanic = load 'hdfs://localhost:9099/user/hive/warehouse/titanic.db/passengers_qualified' 
using PigStorage() as (id:int, survived:int, class:int, 
name:chararray, sex:chararray,age:int, sibsp:int, parch:int, ticket:chararray, 
fare:double, cabin:chararray, embarked:chararray);
```

Raggruppiamo i dati per genere e classe al fine di contare gli imbarcati per ogni tipo. Queste clausole **non eseguono** task YARN sui dati. Agiscono solo sui metadati. Le operazioni di ``dump`` materializzano gli alias generando il piano di esecuzione MapReduce.

```
embarked = group titanic by (sex,class);

embarked_totals = FOREACH embarked GENERATE $0, COUNT(titanic) as embarked:int;
```

Generiamo l'alias dei sopravvissuti e raggruppiamolo per classe e genere.

```
survived = filter titanic by survived == 1;

survived_grouped = group survived by (sex,class);
```

Eseguiamo il join sulla chiave ``(sex, class)`` di imbarcati e sopravvissuti

```
embarked_survived = JOIN survived_grouped BY $0, embarked_totals BY $0;
```

Generiamo infine i conteggi dei sopravvissuti, degli imbarcati, la percentuale e l'età media a partire dal join.

```
survived_counts = FOREACH embarked_survived GENERATE $0, COUNT(survived), $3,
COUNT(survived)*100.0/$3, AVG(survived.age);
```

Analogo con la nostra UDF

```
survived_stats = FOREACH survived_counts GENERATE $0, $1, myfuncs.percentage($1,$2), $4;
```

Salvataggio in HDFS tramite il SER/DE ``PigStorage()``

```
store survived_counts into 'hdfs://localhost:9099/user/pirrone/pig/survived_stats' using PigStorage();


